# Inferencing


In [1]:
%cd /content/drive/MyDrive/Neural-CF

/content/drive/MyDrive/Neural-CF


In [83]:
!python inference.py \
--model_checkpoint './Torch-NCF/checkpoints_neumf_epoch500_l2-0.0000001/pretrain_neumf_factor8neg4_Epoch500_HR0.7381_NDCG0.3776.model' \
--data_dir './Yelp-Dataset/subset/subset_review.json' \
--userId ['-3s52C4zL_DHRK0ULG6qtg'] \
--topK 20

----------Top 20 for [-3s52C4zL_DHRK0ULG6qtg]----------

['QWENxNxNNW2jhQi2akONJA' 'Dty7fGIonmXOmvZhnMAOQQ'
 '_gVyuTRb_6HM-SNtqbpevQ' 'SyAYFxPBjuaHajx9iE7wLQ'
 'KChWMIHxMChTE11ZVsIVZw' 'oHvEgLH6pAkcrPmeR1l3UQ'
 'nn5tvvgFtnoyCocACvXmJQ' '95Ovc1lSnOQwSrjIFhpYrA'
 'XHbaHX2uBGIn3uw37jl7Og' '_C7QiQQc47AOEv4PE3Kong'
 'MuKoTR56s6elHEI2wUkgJA' 'Ct6pupk8Cy6sI2mrIzF7UA'
 'LM54ufrINJWoTN5imV8Etw' 'UFCN0bYdHroPKu6KV5CJqg'
 'rWjFmr0hhi2w8IwMu162-Q' 'h4RaM9E8C0d20Uw-nLR5aw'
 'p1257iwsyKj00D_145O03w' 'y1Z9tymuBGVDZnYZoLk-2Q'
 'ytynqOUb3hjKeJfRj5Tshw' 'of-GVqY0G_UDHXT9Xg_ADQ']
